# Importation des packages & modules nécessaires

In [1]:
import pandas as pd
import random
import yaml
import os
import sys
import requests
from io import StringIO
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

import src.constants as csts

# Headers

In [3]:
# Chargement des headers
with open("headers.yml") as f_headers:
    browser_headers = yaml.safe_load(f_headers)

# Exemple de Header de Firefox
browser_headers

{'Chrome': {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'accept-encoding': 'gzip, deflate, br',
  'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
  'cache-control': 'max-age=0',
  'sec-fetch-dest': 'document',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-user': '?1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'},
 'Edge': {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'accept-encoding': 'gzip, deflate, br',
  'accept-language': 'en-GB,en;q=0.9,en-US;q=0.8',
  'cache-control': 'max-age=0',
  'sec-fetch-dest': 'document',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'none',
  'sec-fetch-user': '?1',
  'upgrade-insecur

# Proxies

In [15]:
proxy_url = "https://free-proxy-list.net/"
response = requests.get(proxy_url)
proxy_list = pd.read_html(StringIO(response.text))[0]
proxy_list["url"] = "http://" + proxy_list["IP Address"] + ":" + proxy_list["Port"].astype(str)
https_proxies = proxy_list.loc[proxy_list["Https"] == "yes", "url"].tolist()
print(len(https_proxies))
https_proxies

11


['http://5.161.212.254:3128',
 'http://68.183.48.146:10006',
 'http://189.240.60.168:9090',
 'http://160.19.94.188:5671',
 'http://18.230.145.231:3128',
 'http://13.48.26.64:8080',
 'http://159.65.176.77:10000',
 'http://20.6.0.172:80',
 'http://68.183.144.115:10000',
 'http://72.10.164.178:10801',
 'http://171.67.71.112:8083']

In [17]:
url = "https://www.societe.com/societe/france-croco-501737761.html"

good_proxies = []

for proxy_url in https_proxies:

    browser_name = random.choice(csts.BROWSERS)
    headers = browser_headers[browser_name]
    proxies = {"http": proxy_url, "https": proxy_url}

    try:
        response = requests.get(url=url, headers=headers, proxies=proxies, timeout=10)
        assert response.status_code == 200
        good_proxies.append(proxy_url)
        print(f"{proxy_url} OK, added to good_proxies set")
    except Exception as e:
        print(f"{proxy_url} NOT OK")
        print(e)

if len(good_proxies) == 1:
    print("Let's continue !")

http://5.161.212.254:3128 NOT OK
HTTPSConnectionPool(host='www.societe.com', port=443): Max retries exceeded with url: /societe/france-croco-501737761.html (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6de746f0d0>: Failed to establish a new connection: [Errno 111] Connection refused')))
http://68.183.48.146:10006 NOT OK
HTTPSConnectionPool(host='www.societe.com', port=443): Max retries exceeded with url: /societe/france-croco-501737761.html (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6de73d2a00>: Failed to establish a new connection: [Errno 111] Connection refused')))
http://189.240.60.168:9090 NOT OK
HTTPSConnectionPool(host='www.societe.com', port=443): Max retries exceeded with url: /societe/france-croco-501737761.html (Caused by SSLError(SSLError(1, '[SSL: KRB5_S_TKT_NYV] unexpected eof while reading (_ssl.c:2622)')))
http://160.1

In [14]:
good_proxies

[]

In [ ]:
url = "https://www.societe.com/societe/france-croco-501737761.html"

req = Request(url, headers={'User-Agent': browser_headers.get("IE").get("User-Agent")})

# Envoi de la requête et récupération de la réponse
webpage = urlopen(req).read()

# Analyse de la réponse HTML à l'aide de BeautifulSoup
contenu_page = bs(webpage, "html.parser")

In [ ]:
contenu_page.findAll(name="table", class_="Table identity mt-16")

In [18]:
response = requests.get(url, headers=browser_headers["Firefox"])
assert response.status_code == 200
html = response.content
soup = bs(html, "html.parser")

In [19]:
soup.find_all(name="table", class_="Table identity mt-16")[0]

<table class="Table identity mt-16" id="rensjur"><tr><td class="arrow">Date création entreprise</td><td class="flex space-between flex-wrap"><div class="TableTexteGauche"><span class="TableTextGenerique"> 17-12-2007 </span><span class="TableTextDetail"><span class="show-mobile-inline-l"> - </span>il y a 16 ans </span></div><span class="TableLienDroite"><a class="Lien with-picto" href="https://www.societe.com/documents-officiels/france-croco-501737761.html#statutshop"><span>Statuts constitutifs</span><i class="material-icons-outlined">chevron_right</i></a></span></td></tr><tr class="Paywall"><td class="arrow" colspan="2"><div class="FicheDonnees__around imrFirst" id="catjur-histo-around"><div class="FicheDonnees" id="catjur-histo" onclick="show_historique_donnees('catjur-histo-voir')"><div class="abonne-compteplus donneesHistoriques voirPlus" id="catjur-histo-voir"><p class="Lien with-picto"><span class="hide-mobile-inline-l">Historique</span><span class="show-mobile-inline-l">Voir l'hi

# Test avec 2 URLs

A l'avenir, ces listes seront remplacées par un fichier excel contenant 2 colonnes : une colonne "Nom" et une colonne "SIREN"

In [ ]:
# Nom "brutes"
liste_nom = ["France Croco", "Wahaus"]
print(">> Liste de noms avant préparation :", liste_nom)

# Noms formatés pour les injecter dans l'URL
liste_nom = [nom.strip().lower().replace(" ", "-") for nom in liste_nom]
print(">> Liste de noms après préparation :", liste_nom)

# Numéros SIREN
liste_numero_siren = ["501737761", "537443228"]
print(">> Numéros SIREN :", liste_numero_siren)

In [ ]:
# Création de la liste des urls
liste_url = [
    csts.BASE_URL + nom + "-" + siren + csts.HTML
    for nom, siren in list(zip(liste_nom, liste_numero_siren))
]

# Affichage de la liste des urls
liste_url

# C TIPAR

In [ ]:
# Instantiation d'une liste vide prête à accueillir les données des entreprises
donnees_entreprises = []

# Pour chaque url
for url in liste_url:
    # Stockage du contenu de la page dans un object BeautifulSoup permettant d'accéder facilement aux éléments de la page
    contenu_page = generate_soup(url=url)

    # Extraction des éléments nécessaires de la page (composant "table" avec la classe "Table identity mt-16")
    contenu_page = contenu_page.findAll(name="table", class_="Table identity mt-16")[0]

    # Extraction du nom de l'entreprise
    nom = contenu_page.find(name="td", class_="break-word").text

    # Extraction de l'adresse
    adresse = contenu_page.find(name="a", class_="Lien secondaire").text.strip()

    # Extraction du numéro SIREN
    numero_siren = contenu_page.find(name="div", id="siren_number").text

    # Extraction du numéro SIRET
    numero_siret = contenu_page.find(name="div", id="siret_number").text

    # Extraction du numéro TVA
    numero_tva = contenu_page.find(name="div", id="tva_number").text

    # Stockage des données d'une entreprise dans un dataframe
    donnees_entreprise = pd.DataFrame(
        [
            {
                "Nom Commerciaux": nom,
                "Adresse": adresse,
                "Numéro SIREN": numero_siren,
                "Numéro SIRET": numero_siret,
                "Numéro TVA": numero_tva,
            }
        ]
    )

    # Ajout du dataframe de l'entreprise à la liste des dataframes des entreprises
    donnees_entreprises.append(donnees_entreprise)

# Concaténation de l'ensemble des dataframes en un seul et unique tableau
donnees_entreprises = pd.concat(objs=donnees_entreprises, axis=0, ignore_index=True)

# ET VOILA

In [ ]:
donnees_entreprises